# Exame de CSI-22 (Programação Orientada a Objetos)
### Grupo: André Diogo, Antônio Gustavo e Lucca Haddad

### Parte 1: Importação de Bibliotecas/Dados

In [61]:
import numpy as np
import pandas as pd

data = pd.read_csv('./digit-recognizer/train.csv')

### Parte 2: Seleção de dados para treino/teste

#### Mudança da estrutura de Pandas para Numpy

#### Aquisição das dimensões

#### Redistribuição dos dados

In [62]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

#### Separação dos dados de teste

#### Normalização do teste

In [ ]:

data_test = data[0:1000].T
Y_test = data_test[0]
X_test = data_test[1:n]
X_test = X_test/255


#### Separação dos dados de treino

#### Normalização do treino

In [ ]:

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train/255

#### Pré-definição do lambda de regressão

In [ ]:
lambda_reg = 0.5

### Parte 3: Definição de funções da Rede Neural

#### Criação inicial dos vetores de pesos e de vieses com valores randômicos

In [63]:
def initializeParameters():
    W1 = np.random.randn(50, 784) * np.sqrt(2/784)
    b1 = np.zeros((50, 1))
    W2 = np.random.randn(10, 50) * np.sqrt(2/50)
    b2 = np.zeros((10, 1))
    return W1, b1, W2, b2

#### Definição da função sigmodal de ativação

In [ ]:
def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

#### Definição da função softmax (exponenciais normalizadas)

In [ ]:
def softmax(Z):
    k = np.exp(Z) / sum(np.exp(Z))
    return k

#### Definição da função de Forward Propagation

In [ ]:
def forwardPropagation(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = sigmoid(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

#### Derivada da função sigmodal

In [ ]:
def sigmoidDerivative(Z):
    sigmoid_Z = sigmoid(Z)
    return sigmoid_Z * (1 - sigmoid_Z)

#### Função para transformação de vetor em One Hot Encoding

In [ ]:
def oneHotEncoding(Y):
    oneHotY = np.zeros((Y.size, Y.max() + 1))
    oneHotY[np.arange(Y.size), Y] = 1
    oneHotY = oneHotY.T
    return oneHotY

#### Backward Propagation para retransmissão da informação para camadas anteriores

In [ ]:
def backwardPropagation(Z1, A1, Z2, A2, W1, W2, X, Y):
    oneHotY = oneHotEncoding(Y)
    m = X.shape[1]
    dZ2 = A2 - oneHotY
    dW2 = (1 / m) * dZ2.dot(A1.T) + (lambda_reg / m) * W2
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * sigmoidDerivative(Z1)
    dW1 = (1 / m) * dZ1.dot(X.T) + (lambda_reg / m) * W1
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

#### Atualização de pesos e de vieses da rede neural

In [ ]:
def updateParameters(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

#### Funções de aquisição do resultado da predição e de determinação de acurácia

In [64]:
def getPredictions(A2):
    return np.argmax(A2, 0)

def getAccuracy(predictions, Y):
    return np.sum(predictions == Y)/Y.size

#### Função para treinamento iterativo do modelo por minimização de perda e retransmissão de informação

In [ ]:
def gradientDescent(X, Y, alpha, iterations, lambda_reg):
    W1, b1, W2, b2 = initializeParameters()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forwardPropagation(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backwardPropagation(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = updateParameters(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if (i % 50 == 0):
            print('Iteration:', i)
            predictions = getPredictions(A2)
            print('Accuracy:', getAccuracy(predictions, Y))
    return W1, b1, W2, b2

#### Treinamento da Rede Neural

In [65]:
W1, b1, W2, b2 = gradientDescent(X_train, Y_train, 0.5, 1000, lambda_reg)

Iteration: 0
Accuracy: 0.09517073170731707
Iteration: 50
Accuracy: 0.8359268292682926
Iteration: 100
Accuracy: 0.8738536585365854
Iteration: 150
Accuracy: 0.8887073170731707
Iteration: 200
Accuracy: 0.8964878048780488
Iteration: 250
Accuracy: 0.9020487804878049
Iteration: 300
Accuracy: 0.9068780487804878
Iteration: 350
Accuracy: 0.9104634146341464
Iteration: 400
Accuracy: 0.9135121951219513
Iteration: 450
Accuracy: 0.9163902439024391
Iteration: 500
Accuracy: 0.9187560975609756
Iteration: 550
Accuracy: 0.9210487804878049
Iteration: 600
Accuracy: 0.9229024390243903
Iteration: 650
Accuracy: 0.9246585365853659
Iteration: 700
Accuracy: 0.9269024390243903
Iteration: 750
Accuracy: 0.9288536585365854
Iteration: 800
Accuracy: 0.9308048780487805
Iteration: 850
Accuracy: 0.9323414634146342
Iteration: 900
Accuracy: 0.9337073170731708
Iteration: 950
Accuracy: 0.9350975609756098


#### Função para predição de um vetor desconhecido e validação com o vetor de testes

In [66]:
def makePredictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forwardPropagation(W1, b1, W2, b2, X)
    predictions = getPredictions(A2)
    return predictions

predicted = makePredictions(X_test, W1, b1, W2, b2)
getAccuracy(predicted, Y_test)

0.937

#### Importação dos dados para predição

In [67]:
sub = pd.read_csv('./digit-recognizer/test.csv')
sub = np.array(sub)
sub = sub.T
sub = sub/255

#### Transformação para CSV e submissão

In [68]:
pred_sub = makePredictions(sub, W1, b1, W2, b2)
submission = pd.read_csv('./digit-recognizer/sample_submission.csv')
submission['Label'] = pred_sub
submission.to_csv('pred.csv', index=False)